In [1]:
import json
import xmltodict

In [2]:
# converts time string to seconds

def convertTime(curTime):
    curTime = curTime[:8]
    parts = curTime.split(':')
    return int(parts[0])*3600 + int(parts[1])*60 + int(parts[2])

In [105]:
# reading xml file and un-marshalling to python object
# converting the time to seconds
# calculating slack time

xml_file = open("data4.lwf")
data = xmltodict.parse(xml_file.read())["lease-workload"]
xml_file.close()

maxNodes = int(data["site"]["nodes"]["node-set"]["@numnodes"])
strval = ""
requests = []
for request in data["lease-requests"]["lease-request"]:
    temp = {}
    temp["id"] = request["lease"]["@id"]
    temp["arrival"] = convertTime(request["@arrival"])
    temp["nodes"] = int(request["lease"]["nodes"]["node-set"]["@numnodes"])
    temp["start"] = convertTime(request["lease"]["start"]["exact"]["@time"])
    temp["duration"] = convertTime(request["lease"]["duration"]["@time"])
    temp["deadline"] = convertTime(request["lease"]["deadline"]["@time"])
    temp["slack"] = round(
        (temp["deadline"] - temp["start"]) / temp["duration"], 2)
    requests.append(temp)

In [106]:
requests = sorted(requests, key=lambda d: d['arrival'])

In [107]:
def haizea(requests):
    temp = []
    for req in requests:
        if(req["slack"]<=1.1):
            printf("Request with ID = %d has been rejected due to slack time <=1.1", req["id"])
            continue
        else:
            temp.append(req)

In [108]:
def check(sch,start,end):
    for i in range(len(sch)-1):
        if(sch[i][1]<=start and end<=sch[i+1][0]):
            return True
    return False


In [109]:
def findSlot(req, sch):
    startTime = req["start"]
    deadline = req["deadline"]
    duration = req["duration"]
    nodes = req["nodes"]
    id=req["id"]
    found=False

    for i in range(len(sch)):
        for j in range(len(sch[i])-1):
            if(sch[i][j][1]>=startTime or startTime<=sch[i][j+1][0]):    #checkling if startime is grater than or equal to start time or less than next lease endtime  
                st=max(startTime,sch[i][j][1])                              #start time is max of start time and end time of previous end time
                if(st+duration<=sch[i][j+1][0] and st+duration<=deadline):      

                    available_nodes=[]                                  #available node for the scheduling for given start and duration

                    for x in range(len(sch)):
                        if check(sch[x],st,st+duration):                #checking the nodes for the given start and duration
                            available_nodes.append(x)
                    
                    if len(available_nodes)>=nodes: 
                        
                        tmp=[st,st+duration,id]
                        for x in range(nodes):

                            sch[available_nodes[x]].append(tmp)
                            sch[available_nodes[x]].sort()
                        found=True
                        break
        
        if found:
            return True
    return False             

            

In [110]:
def reschedule(req,sc):

    dup = []
    count=0
    st=[0,0,0] 
    en=[1e9,1e9,1e9]

    for i in range(maxNodes):
        tmp=[]
        dup.append(tmp)
    nsl=[]
    st=req["start"]
    print(str(req))
    for i in range(len(sc)):
        for j in range(len(sc[i])):

            if(st<=sc[i][j][0]):
                nsl.append(sc[i][j])
            else:
                dup[i].append(sc[i][j])

    nsld=[]
    nsld.append(req)
    for i in range(len(requests)):
        for j in range(len(nsl)):

            if(requests[i]["id"]==nsl[j][2]):
                nsld.append(requests[i])
    
    print(type(nsld[0]))
    for i in range(len(nsld)):

        if not findSlot(nsld[i],dup):
            
            return sc,1
        
    return dup,0


In [111]:
sc = []
count=0
st=[0,0,0] 
en=[1e9,1e9,1e9]

for i in range(maxNodes):
    tmp=[st,en]
    sc.append(tmp)


for i in range(len(requests)):
    if not findSlot(requests[i],sc):
        sc,didnt=reschedule(requests[i],sc)
        count+=didnt  

print("Total Leases:", len(requests))
print("Total Leases Scheduled:", len(requests) - count)
print("Scheduled Percentage:", ((len(requests) - count)/ len(requests))*100)
print("Not possible to schedule", count)
    


{'id': '2', 'arrival': 43210, 'nodes': 2, 'start': 43800, 'duration': 7200, 'deadline': 57600, 'slack': 1.92}
<class 'dict'>
{'id': '6', 'arrival': 43250, 'nodes': 4, 'start': 64800, 'duration': 3600, 'deadline': 70200, 'slack': 1.5}
<class 'dict'>
{'id': '10', 'arrival': 43290, 'nodes': 4, 'start': 72000, 'duration': 1800, 'deadline': 77400, 'slack': 3.0}
<class 'dict'>
{'id': '11', 'arrival': 43300, 'nodes': 4, 'start': 79200, 'duration': 1800, 'deadline': 82800, 'slack': 2.0}
<class 'dict'>
{'id': '12', 'arrival': 43310, 'nodes': 4, 'start': 82800, 'duration': 1800, 'deadline': 86400, 'slack': 2.0}
<class 'dict'>
{'id': '17', 'arrival': 43360, 'nodes': 4, 'start': 90000, 'duration': 1800, 'deadline': 93600, 'slack': 2.0}
<class 'dict'>
{'id': '19', 'arrival': 43380, 'nodes': 4, 'start': 97200, 'duration': 3600, 'deadline': 108000, 'slack': 3.0}
<class 'dict'>
{'id': '21', 'arrival': 43400, 'nodes': 4, 'start': 108000, 'duration': 3600, 'deadline': 115200, 'slack': 2.0}
<class 'dict'

In [102]:
sc[0]

[[0, 0, 0],
 [43800, 54000, '1'],
 [54000, 55200, '29'],
 [57600, 60000, '2'],
 [60000, 61800, '3'],
 [61800, 63600, '4'],
 [65400, 66600, '5'],
 [66600, 68400, '6'],
 [68400, 70800, '7'],
 [70800, 71400, '24'],
 [71400, 78600, '8'],
 [82200, 83400, '9'],
 [83400, 85800, '10'],
 [87000, 90600, '11'],
 [90600, 93000, '31'],
 [93000, 95400, '12'],
 [95400, 97800, '32'],
 [97800, 100800, '13'],
 [1000000000.0, 1000000000.0, 1000000000.0]]

In [103]:
sc[1]

[[0, 0, 0],
 [43800, 54000, '1'],
 [57600, 60000, '2'],
 [60000, 61800, '3'],
 [61800, 63600, '4'],
 [63600, 66000, '23'],
 [66600, 68400, '6'],
 [68400, 70800, '7'],
 [70800, 71400, '24'],
 [71400, 78600, '8'],
 [82200, 83400, '9'],
 [83400, 85800, '10'],
 [87000, 90600, '11'],
 [90600, 93000, '31'],
 [93000, 95400, '12'],
 [95400, 97800, '32'],
 [97800, 100800, '13'],
 [1000000000.0, 1000000000.0, 1000000000.0]]

In [104]:
sc[2]

[[0, 0, 0],
 [43800, 54000, '1'],
 [60000, 61800, '3'],
 [61800, 63600, '4'],
 [63600, 66000, '23'],
 [66600, 67200, '21'],
 [68400, 70800, '7'],
 [70800, 71400, '24'],
 [71400, 78600, '8'],
 [82200, 83400, '27'],
 [83400, 85800, '10'],
 [87000, 90600, '11'],
 [93000, 95400, '12'],
 [95400, 97800, '32'],
 [97800, 100800, '33'],
 [1000000000.0, 1000000000.0, 1000000000.0]]

In [72]:
data

{'@name': 'deadline10',
 'description': 'First lease does not end prematurely.\nSecond and third lease end prematurely.\nSecond and third lease start at 00:15:00.00 and 00:20:00.00',
 'site': {'resource-types': {'@names': 'CPU Memory'},
  'nodes': {'node-set': {'@numnodes': '3',
    'res': [{'@type': 'CPU', '@amount': '100'},
     {'@type': 'Memory', '@amount': '1024'}]}}},
 'lease-requests': {'lease-request': [{'@arrival': '12:00:00.00',
    'lease': {'@id': '1',
     '@preemptible': 'true',
     'nodes': {'node-set': {'@numnodes': '2',
       'res': [{'@amount': '100', '@type': 'CPU'},
        {'@amount': '1024', '@type': 'Memory'}]}},
     'start': {'exact': {'@time': '12:10:00.00'}},
     'duration': {'@time': '02:00:00.00'},
     'deadline': {'@time': '16:00:00.00'},
     'software': {'disk-image': {'@id': 'foobar1.img', '@size': '1024'}}}},
   {'@arrival': '12:00:10.00',
    'lease': {'@id': '2',
     '@preemptible': 'true',
     'nodes': {'node-set': {'@numnodes': '2',
       'r

In [76]:
data['site']['nodes']['node-set']['@numnodes']

'4'